In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=dbd8dd5bbe83fdee9d2c2a0ba776be30d336d3ab5c539be7dead9b73303ace6f
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [3]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Food DF').getOrCreate()

In [4]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [5]:
df = spark.read.csv("indian_food.csv",header=True,inferSchema=True)
df.show()

+--------------+--------------------+----------+---------+---------+--------------+-------+-------------+------+
|          name|         ingredients|      diet|prep_time|cook_time|flavor_profile| course|        state|region|
+--------------+--------------------+----------+---------+---------+--------------+-------+-------------+------+
|    Balu shahi|Maida flour, yogu...|vegetarian|       45|       25|         sweet|dessert|  West Bengal|  East|
|        Boondi|Gram flour, ghee,...|vegetarian|       80|       30|         sweet|dessert|    Rajasthan|  West|
|Gajar ka halwa|Carrots, milk, su...|vegetarian|       15|       60|         sweet|dessert|       Punjab| North|
|        Ghevar|Flour, ghee, kewr...|vegetarian|       15|       30|         sweet|dessert|    Rajasthan|  West|
|   Gulab jamun|Milk powder, plai...|vegetarian|       15|       40|         sweet|dessert|  West Bengal|  East|
|        Imarti|Sugar syrup, lent...|vegetarian|       10|       50|         sweet|dessert|  Wes

In [6]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- ingredients: string (nullable = true)
 |-- diet: string (nullable = true)
 |-- prep_time: integer (nullable = true)
 |-- cook_time: integer (nullable = true)
 |-- flavor_profile: string (nullable = true)
 |-- course: string (nullable = true)
 |-- state: string (nullable = true)
 |-- region: string (nullable = true)



In [19]:
# 1.Find out how many unique dishes are present.

df1 = df.select(col('name')).distinct().count()
df1

255

In [20]:
# 2.Which state has more dishes?

df2 = df.select(col('state')).groupBy('state').count()
df2.select('state','count').filter(col('count')==df2.agg({'count':'max'}).collect()[0][0]).show()

+-------+-----+
|  state|count|
+-------+-----+
|Gujarat|   35|
+-------+-----+



In [15]:
# 3.How many dishes from state Karnataka?

df3 = df.where(col('state')=='Karnataka').count()
df3

6

In [25]:
# 4.List number of unique regions.

df4 = df.select(col('region')).distinct().show()
df4

+----------+
|    region|
+----------+
|        -1|
|     South|
|   Central|
|      East|
|      West|
|North East|
|     North|
|      NULL|
+----------+



In [26]:
# 5.Count number of dishes from each region.

df5 = df.select(col('region')).groupBy('region').count().show()
df5

+----------+-----+
|    region|count|
+----------+-----+
|        -1|   13|
|      NULL|    1|
|     South|   59|
|   Central|    3|
|      East|   31|
|      West|   74|
|North East|   25|
|     North|   49|
+----------+-----+



In [27]:
# 6. List unique 'flavor_profile' and 'course'

df6 = df.select(col('flavor_profile'),col('course')).distinct().show()
df6

+--------------+-----------+
|flavor_profile|     course|
+--------------+-----------+
|        bitter|      snack|
|         spicy|    starter|
|          sour|main course|
|            -1|      snack|
|            -1|main course|
|         sweet|main course|
|        bitter|main course|
|         spicy|      snack|
|         sweet|    dessert|
|         spicy|main course|
+--------------+-----------+



In [21]:
# 7.Which state has more 'main course'?

df7=df.select('state').groupBy('state').count()
df7.select('state','count').where(col('count')==df7.agg({'count':'max'}).collect()[0][0]).show()

+-------+-----+
|  state|count|
+-------+-----+
|Gujarat|   35|
+-------+-----+



In [23]:
# 8.Give the %of dishes from each region

total=df.count()
df.select('region').groupBy('region').count().withColumn('percentage',col('count')/ total*100).show()

+----------+-----+-------------------+
|    region|count|         percentage|
+----------+-----+-------------------+
|        -1|   13|  5.098039215686274|
|      NULL|    1|0.39215686274509803|
|     South|   59| 23.137254901960784|
|   Central|    3| 1.1764705882352942|
|      East|   31| 12.156862745098039|
|      West|   74|  29.01960784313726|
|North East|   25|  9.803921568627452|
|     North|   49| 19.215686274509807|
+----------+-----+-------------------+



In [24]:
# 9.List the states which has more dishes from each region.

df9=df.select('state','region').groupBy('state','region').count()
df9.select('state','region','count').filter(col('count')==df9.agg({'count':'max'}).collect()[0][0]).show()

+-------+------+-----+
|  state|region|count|
+-------+------+-----+
|Gujarat|  West|   35|
+-------+------+-----+

